In [ ]:
import geopandas as gpd
from shapely.geometry import box
import matplotlib.pyplot as plt

In [ ]:
# Loading shapefiles from the current working folder
osm_road_network = gpd.read_file(r"C:\Users\srini\OneDrive\Documents\ArcGIS\Projects\Term Project\Atlanta Roadways\Atlanta_Roadways_line.shp")
atlanta_shapefile = gpd.read_file(r"C:\Users\srini\OneDrive\Documents\ArcGIS\Projects\Term Project\Geographic boundaries\Geographic_boundaries,_ACS_2022.shp")
traffic_counts = gpd.read_file(r"C:\Users\srini\OneDrive\Documents\ArcGIS\Projects\Term Project\Traffic Statistics\features.shp")


In [ ]:
# Align CRS for all datasets
atlanta_shapefile = atlanta_shapefile.to_crs(osm_road_network.crs)
traffic_counts = traffic_counts.to_crs(osm_road_network.crs)

In [ ]:
# Define the bounding box from the extent of the OSM road network
osm_bbox = osm_road_network.total_bounds  # [minx, miny, maxx, maxy]
osm_boundary = gpd.GeoDataFrame(geometry=[box(*osm_bbox)], crs=osm_road_network.crs)

# Clip datasets to the OSM bounding box
atlanta_clipped = gpd.clip(atlanta_shapefile, osm_boundary)
traffic_counts_clipped = gpd.clip(traffic_counts, osm_boundary)
road_network_clipped = osm_road_network

In [ ]:
# Spatial join between traffic counts and road network
traffic_with_roads = gpd.sjoin(road_network_clipped, traffic_counts_clipped, how="inner", predicate="intersects")


In [ ]:
# Plot the road network, traffic counts, and boundary
fig, ax = plt.subplots(figsize=(12, 8))
osm_boundary.boundary.plot(ax=ax, color="blue", linewidth=1, label="OSM Boundary")
road_network_clipped.plot(ax=ax, color="grey", linewidth=0.5, label="Road Network")
traffic_counts_clipped.plot(ax=ax, color="red", markersize=10, label="Traffic Counts")

# Customize the map
ax.set_title("Preliminary Traffic Visualization: Atlanta and Suburbs", fontsize=14)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.legend()
plt.show()

In [ ]:
#We have completed the spatial preprocessing phase, including clipping datasets, aligning CRS, and preparing clean datasets for analysis. Moving forward to spatial analysis, we will identify traffic congestion hotspots, perform proximity analysis with road infrastructure, and extract statistical summaries to reveal key patterns.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import numpy as np
import pandas as pd

# Ensure matplotlib inline if using Jupyter
%matplotlib inline


In [ ]:
# Spatial join of traffic counts with the road network to associate traffic data with the road segments
traffic_with_roads = gpd.sjoin(traffic_counts, osm_road_network, how="inner", op="intersects")

# Analyze the traffic counts along these segments, filtering by traffic volume
high_traffic_threshold = 10000  # Set traffic threshold to 1500
high_traffic_areas = traffic_with_roads[traffic_with_roads['aadt'] > high_traffic_threshold]

# Plot the high-traffic areas
ax = atlanta_shapefile.plot(figsize=(10, 10), color='lightgray')
high_traffic_areas.plot(ax=ax, color='red', markersize=10, label='High Traffic Areas')
osm_road_network.plot(ax=ax, color='gray', linewidth=0.5, label='Road Network')
plt.legend()
plt.title('High Traffic Areas in Atlanta')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()


In [ ]:
# Convert the 'aadt' column to numeric, coercing any errors to NaN
traffic_counts['aadt'] = pd.to_numeric(traffic_counts['aadt'], errors='coerce')

# Now try calculating the threshold again
threshold = traffic_counts['aadt'].quantile(0.90)  # Define a threshold for hotspots
hotspots = traffic_counts[traffic_counts['aadt'] > threshold]

# Save hotspots for further use
hotspots.to_file("traffic_hotspots.shp")


In [ ]:
# Convert the 'aadt' column to numeric, coercing any errors to NaN
traffic_counts_clipped['aadt'] = pd.to_numeric(traffic_counts_clipped['aadt'], errors='coerce')

# Now try calculating the threshold again
threshold = traffic_counts_clipped['aadt'].quantile(0.90)  # Define a threshold for hotspots
hotspots = traffic_counts_clipped[traffic_counts_clipped['aadt'] > threshold]

# Save hotspots for further use
hotspots.to_file("traffic_hotspots.shp")

In [ ]:
# Example: Assume "road_network" and "hotspots" are in the same CRS
hotspots['nearest_road_distance'] = hotspots.geometry.apply(
    lambda point: road_network_clipped.distance(point).min()
)

# Save proximity results
hotspots.to_file("hotspots_with_proximity.shp")

In [ ]:
# Summary of traffic volumes
volume_stats = traffic_counts['aadt'].describe()
print(volume_stats)

# Aggregated stats for hotspots
hotspot_stats = hotspots['aadt'].describe()
print(hotspot_stats)

# Save summaries to CSV if needed
volume_stats.to_csv("traffic_volume_stats.csv")
hotspot_stats.to_csv("hotspot_volume_stats.csv")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
atlanta_clipped.plot(ax=ax, edgecolor='blue', linewidth=1, label="Boundary")
road_network_clipped.plot(ax=ax, color='white', linewidth=0.5, label="Road Network")
traffic_counts_clipped.plot(ax=ax, color='red', markersize=5, label="Traffic Counts")
hotspots.plot(ax=ax, color='yellow', markersize=10, label="Hotspots")

# Add legend and titles
plt.legend()
plt.title("Traffic Congestion Hotspots")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()